SPARK 19 - Para cada categoría, indicar cuál es la aplicación que tiene mayor cantidad de reviews con sentimiento negativo (⭐⭐)

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [4]:
# Crea la sesion de SPARK
spark = SparkSession.builder.getOrCreate()

# Crea el SPARK content
sc = spark.sparkContext

In [5]:
sqlContext = SQLContext(sc)
infoplay = sqlContext.read.csv("/content/drive/MyDrive/Colab Notebooks/TP1-VISU/googleplaystore.csv", header=True)
inforeviews = sqlContext.read.csv("/content/drive/MyDrive/Colab Notebooks/TP1-VISU/googleplaystore_user_reviews.csv", header=True)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [6]:
appinfoplayRDD = infoplay.rdd
appinforeviewsRDD = inforeviews.rdd

In [ ]:
categorias = appinfoplayRDD.map(lambda x: x.Category).distinct()
categorias.collect()

Quito el valor erroneo visualizado en la columna categorias de mi rdd y tambien me quedo solo con las opiniones negativas del otro rdd. Luego, combino ambos RDD y me quedo con los elementos que coincidan entre ambas partas

In [8]:
resultadoRDD = appinforeviewsRDD.filter(lambda x: x.Sentiment == 'Negative').map(lambda x: (x.App, 1)).leftOuterJoin(
    appinfoplayRDD.filter(lambda x: x.Category != '1.9').map(lambda x: (x.App, x.Category)))

Se muestran las diferentes categorias junto con la cantidad de reviews que tiene la app con mas reviews negativas de la categoria respectiva

In [9]:
resultadoRDD.map(lambda x: (x[1][1], (x[0],1) ) ).reduceByKey(lambda x,y: (x[0] , x[1]+y[1])).filter(lambda x: (x[0] is not None)).collect()

[('MEDICAL', ('1800 Contacts - Lens Store', 326)),
 ('NEWS_AND_MAGAZINES', ('ARY NEWS URDU', 333)),
 ('TOOLS', ('AT&T Smart Wi-Fi', 135)),
 ('SOCIAL', ('All Social Networks', 345)),
 ('ENTERTAINMENT', ('Animal Planet GO', 450)),
 ('FINANCE', ('BBVA Compass Banking', 263)),
 ('COMMUNICATION', ('CIA - Caller ID & Call Blocker', 152)),
 ('EVENTS', ('DroidAdmin for Android - Advice', 12)),
 ('FAMILY', ('A Call From Santa Claus!', 1175)),
 ('PHOTOGRAPHY', ('Adobe Photoshop Express:Photo Editor Collage Maker', 172)),
 ('LIFESTYLE', ('Anime Avatar Creator: Make Your Own Avatar', 81)),
 ('LIBRARIES_AND_DEMO', ('Apk Installer', 50)),
 ('EDUCATION', ('Babbel – Learn Spanish', 192)),
 ('PARENTING', ('Baby Monitor', 22)),
 ('MAPS_AND_NAVIGATION', ('Czech Public Transport IDOS', 7)),
 ('COMICS', ('DC Comics', 2)),
 ('BEAUTY', ('Dresses Ideas & Fashions +3000', 41)),
 ('GAME', ('1LINE – One Line with One Touch', 5332)),
 ('DATING', ('2RedBeans', 332)),
 ('HEALTH_AND_FITNESS', ('30 Day Fitness Challe